# [Excel Charts](https://www.osgeo.cn/openpyxl/charts/introduction.html#chart-types)

## 轴最小值和最大值

```python
chart = ScatterChart()
chart1.title = 'Full Axes'
chart1.x_axis.title = 'x'
chart1.y_axis.title = 'y'
chart1.legend = None

# 设置x、y轴坐标范围 (设置轴范围实际上相当于限定的显示数据的范围)
chart1.x_axis.scaling.min = 0
chart1.x_axis.scaling.max = 11
chart1.y_axis.scaling.min = 0
chart1.y_axis.scaling.max = 1.5
```

## 轴方向
> 轴可以`正常`或`反向`显示，轴方向由属性`orientation`控制，值为`minMax`时正常方向显示，为`maxMin`时反方向显示

```python
chart1.x_axis.scaling.orientation = 'maxMin'
chart1.y_axis.scaling.orientation = 'minMax'
```

## 添加第二个轴
> 添加第二个轴实际上涉及到创建第二个图表，该图表与第一个图表共享一个公共`X`轴，但是有独立的`Y`轴

In [ ]:
from openpyxl import Workbook
from openpyxl.chart import (
    LineChart,
    BarChart,
    Reference,
    Series,
)

wb = Workbook()
ws = wb.active

rows = [
    ['Aliens', 2, 3, 4, 5, 6, 7],
    ['Humans', 10, 40, 50, 20, 10, 50],
]

for row in rows:
    ws.append(row)

# Create first chart    
c1 = BarChart()
# 当 max_row 为空时, 默认 max_row = min_row; 对于 max_col 同理
v1 = Reference(ws, min_col=1, min_row=1, max_col=7)
# 由于数据以行的形式存储，故需要设置 from_rows=True
c1.add_data(v1, titles_from_data=True, from_rows=True)

c1.x_axis.title = 'Days'
c1.y_axis.title = 'Aliens'
c1.y_axis.majorGridlines = None
c1.title = 'Survey results'

# Create second chart
c2 = LineChart()
v2 = Reference(ws, min_col=1, min_row=2, max_col=7)
c2.add_data(v2, titles_from_data=True, from_rows=True)
c2.y_axis.axId = 200
c2.y_axis.title = "Humans"

# 默认情况下，`y`轴都位于左边 (min)
c1.y_axis.crosses = "max"     # C1 图形的 y 轴移到右边
# C2.y_axis.crosses = "max"    # C2 图形的 y 轴移到右边
c1 += c2

ws.add_chart(c1, "D4")
wb.save("secondary.xlsx")

## 尺寸和位置
图表必须放置在其容器中；`x` 和 `y` 用于调整位置；`w` 和 `h` 用于调整尺寸；  
+ `x` 是从左边开始的水平位置  
+ `y` 是从顶部开始的垂直位置
+ `w` 是图表相对于其容器的宽度
+ `h` 是图表相对于其容器的高度

## 图例(legend)布局
图例的位置可以通过参数设置： 默认值为 `r`
+ `r` 表示右 (right)
+ `l` 表示左 (left)
+ `t` 表示上 (top)
+ `b` 表示下 (buttom)
+ `tr` 表示右上 (top-right)

In [ ]:
from openpyxl import Workbook, load_workbook
from openpyxl.chart import ScatterChart, Series, Reference
from openpyxl.chart.layout import Layout, ManualLayout

wb = Workbook()
ws = wb.active

rows = [
    ['Size', 'Batch 1', 'Batch 2'],
    [2, 40, 30],
    [3, 40, 25],
    [4, 50, 30],
    [5, 30, 25],
    [6, 25, 35],
    [7, 20, 40],
]

for row in rows:
    ws.append(row)

ch1 = ScatterChart()

# x轴坐标点
xvalues = Reference(ws, min_col=1, min_row=2, max_row=7)
for i in range(2, 4):
    values = Reference(ws, min_col=i, min_row=1, max_row=7)
    series = Series(values, xvalues, title_from_data=True)
    ch1.series.append(series)

ch1.title = "Default layout"
ch1.style = 13
ch1.x_axis.title = 'Size'
ch1.y_axis.title = 'Percentage'
ch1.legend.position = 'r'

ws.add_chart(ch1, "B10")

from copy import deepcopy

# Half-size chart, bottom right
ch2 = deepcopy(ch1)
ch2.title = "Manual chart layout"
ch2.legend.position = "tr"
ch2.layout=Layout(
    manualLayout=ManualLayout(
        x=0.25, y=0.25,
        h=0.5, w=0.5,
    )
)
ws.add_chart(ch2, "H10")

# Half-size chart, centred
ch3 = deepcopy(ch1)
ch3.layout = Layout(
    ManualLayout(
    x=0.25, y=0.25,
    h=0.5, w=0.5,
    xMode="edge",
    yMode="edge",
    )
)
ch3.title = "Manual chart layout, edge mode"
ws.add_chart(ch3, "B27")

# Manually position the legend bottom left
ch4 = deepcopy(ch1)
ch4.title = "Manual legend layout"
ch4.legend.layout = Layout(
    manualLayout=ManualLayout(
        yMode='edge',
        xMode='edge',
        x=0, y=0.9,
        h=0.1, w=0.5
    )
)

ws.add_chart(ch4, "H27")
wb.save("chart_layout.xlsx")

# 条形图 (barChart)

> 通过设置参数`type`的值为[`col`, `bar`] 可以将条形图切换为`垂直` 或 `水平` 形状   
> 如果使用`堆叠图表`时，需要将参数`overlap`设置为`100`

In [ ]:
from openpyxl import Workbook
from openpyxl.chart import BarChart, Series, Reference

wb = Workbook(write_only=True)
ws = wb.create_sheet()

rows = [
    ('Number', 'Batch 1', 'Batch 2'),
    (2, 10, 30),
    (3, 40, 60),
    (4, 50, 70),
    (5, 20, 10),
    (6, 10, 40),
    (7, 50, 30),
]


for row in rows:
    ws.append(row)

chart1 = BarChart()
# 设置为垂直条形图
chart1.type = "col"
chart1.style = 10
chart1.title = "Bar Chart"
chart1.y_axis.title = 'Test number'
chart1.x_axis.title = 'Sample length (mm)'
# 设置图示数据 (此处选择了两列数据)
data = Reference(ws, min_col=2, min_row=1, max_row=7, max_col=3)
chart1.add_data(data, titles_from_data=True)
# 设置分类
cats = Reference(ws, min_col=1, min_row=2, max_row=7)
chart1.set_categories(cats)
chart1.shape = 4
ws.add_chart(chart1, "A10")

from copy import deepcopy

chart2 = deepcopy(chart1)
chart2.style = 11
# 设置为水平条形图
chart2.type = "bar"
chart2.title = "Horizontal Bar Chart"
ws.add_chart(chart2, "G10")


chart3 = deepcopy(chart1)
chart3.type = "col"
chart3.style = 12
# 设置为堆叠图
chart3.grouping = "stacked"
chart3.overlap = 100
chart3.title = 'Stacked Chart'
ws.add_chart(chart3, "A27")


chart4 = deepcopy(chart1)
chart4.type = "bar"
chart4.style = 13
# 设置为百分比堆叠图
chart4.grouping = "percentStacked"
chart4.overlap = 100
chart4.title = 'Percent Stacked Chart'
ws.add_chart(chart4, "G27")

wb.save("bar.xlsx")